# 2022-12-22 Grid Search (with sklearn)
### 개인 공부 정리전용

**Motivation:**  
Classifier를 만들어야하는데, Deep learning을 사용하기에는 데이터의 양이 너무 작았다.  
그래서 SVM과 같이 sklearn에 있는 ML 모델들을 사용하고자 하였는데, parameter tuning을 다양한 경우의 수에 대해서 하기는 일일이 작업하는 것이 힘들고 시간이 오래 걸렸다.  
그래서 이를 구현한 함수가 있는지 서칭을 하였는데, sklearn.model_selection에 GridSearchCV라는 함수가 구현되어 있었다.

`-` XGBoost와 SVM을 가지고 GridSearchCV 함수를 사용해보았다.  
코드 출처: https://gist.github.com/statcompute/d97c42361a0a84e4ade02ed3fe667ca5

처음 보는 함수여서 잘못된 부분이 있을 수 있음. 

### `-` library import

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [62]:
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd


`-` Toy data

In [52]:
data = pd.DataFrame({"X1":np.arange(0,10,0.1), "X2":np.arange(0,10,0.1)**2})
label = [0]*50 + [1]*50

`-` SVM에 grid search를 적용해본다.

In [70]:
mdl = svm.SVC(probability = True, random_state = 1) # SVM 모델 선언
score = make_scorer(log_loss) # 잘은 모르겠지만 make_scorer는 gridsearch를 사용할 때 같이 사용하는 함수 같다. sklearn 페이지의 example에도 나와있음.https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html
# make_scorer를 빼고 사용하면 split_test_score 부분의 값들이 nan을 ㅗ채워졌다.

# 탐색하고자 하는 parameter 범위 정의
grid_list = {'kernel': ['linear'], "C": np.arange(1, 5, 2),
             'kernel': ['rbf'], "C":np.arange(1, 5, 2), 
             "gamma": np.arange(0.01, 0.05, 0.02)}

# Grid search 진행
grid_search = GridSearchCV(mdl, param_grid = grid_list, cv = 5, scoring = score) 
grid_search.fit(data, label) 
grid_search.cv_results_

{'mean_fit_time': array([0.00203576, 0.0015954 , 0.00102077, 0.00083704]),
 'std_fit_time': array([1.25728111e-03, 5.08748480e-04, 3.78581678e-05, 1.02675208e-03]),
 'mean_score_time': array([0.00118799, 0.00079107, 0.00099869, 0.        ]),
 'std_score_time': array([3.84402132e-04, 3.96022805e-04, 3.32899278e-06, 0.00000000e+00]),
 'param_C': masked_array(data=[1, 1, 3, 3],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[0.01, 0.03, 0.01, 0.03],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'rbf', 'rbf', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'},
  {'C': 1, 'gamma': 0.03, 'kernel': 'rbf'},
  {'C': 3, 'gamma': 0.01, 'kernel': 'rbf'},
  {'C': 3, 'gamma': 0.03, 'kernel': 'rbf'}],
 'split

`-` 위의 grid search에서 나온 최적의 파라메터와 최고 score

In [66]:
grid_search.best_params_, grid_search.best_score_

({'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}, 3.453917619362579)

grid_search.best_params_ 를 사용하면 최적의 파라미터를 준다고 한다.  
grid_search.best_score_ 를 사용하면 최고 점수를 준다고 한다.
- 출처: https://wpaud16.tistory.com/65